## **기본 텐서플로우의 케라스 컴포넌트를 이용**

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
import numpy as np

a_in = np.array([[1,2,3]])
layer = Dense(units=3, activation='sigmoid')
a_out = layer(a_in)
print(a_out)

tf.Tensor([[0.27265733 0.10661475 0.5188296 ]], shape=(1, 3), dtype=float32)


In [5]:
layer

<Dense name=dense, built=True>

>Dense 레이어를 생성한 직후, 아직 학습이 진행되지 않았고 명시적으로 가중치와 바이어스를 초기화하지 않은 경우,  
>텐서플로는 자동으로 무작위 초기화를 수행

In [11]:
layer.kernel.numpy() # (입력 차원, 출력 차원) shape의 가중치 행렬 (cf. 입력과 곱해지는 메인 가중치 행렬을 의미, Keras의 네이밍 컨벤션)

array([[-0.5258627 , -0.7971599 ,  0.05282426],
       [ 0.6406555 , -0.10517836,  0.21609545],
       [-0.57887673, -0.37275982, -0.13655376]], dtype=float32)

In [12]:
layer.bias.numpy()  # (출력 차원,) shape의 바이어스 벡터

array([0., 0., 0.], dtype=float32)

In [14]:
layer.weights       # [kernel, bias] 두 개의 텐서가 포함된 리스트

[<Variable path=dense/kernel, shape=(3, 3), dtype=float32, value=[[-0.5258627  -0.7971599   0.05282426]
  [ 0.6406555  -0.10517836  0.21609545]
  [-0.57887673 -0.37275982 -0.13655376]]>,
 <Variable path=dense/bias, shape=(3,), dtype=float32, value=[0. 0. 0.]>]

## **직접 Dense Layer를 코딩해보기**

### **벡터화 후 병렬 연산 적용**

In [9]:
def dense(a_in, W, b, g):

  z = np.dot(a_in, W) + b
  A = g(z)
  return A

def sigmoid(logit):
  # activation = 1/1+exp(-x)
  activation = 1 / 1 + np.exp(-logit)
  return activation

W = np.array([[1,2,3],
              [3,4,6],
              [7,8,9]])
b = [-1, -2, -5]
dense(a_in, W, b, g=sigmoid)

array([[1., 1., 1.]])

### **순차 연산**

직접 세팅한 값으로 W와 b가 할당되어 있는 상황이라고 가정. (원래는 랜덤 초기화)

In [20]:
def dense0(a_in, W, b, g):
  units = W.shape[1]
  a_out = np.zeros(units)
  print(a_out)

  for i in range(units):
    print(f"\n현재 유닛은 {i}번째")
    # z = np.dot(a_in, W[i]) + b[i]
    z = np.dot(a_in, W[:,i]) + b[i]
    A = g(z)
    print(A)
    a_out[i] = A

  return a_out

dense0(a_in, W, b, g=sigmoid)

[0. 0. 0.]

현재 유닛은 0번째
[1.]

현재 유닛은 1번째
[1.]

현재 유닛은 2번째
[1.]


<ipython-input-20-7ba017206774>:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  a_out[i] = A


array([1., 1., 1.])

#### *가중치 인덱싱(열) 간과*

In [22]:
def dense0(a_in, W, b, g):
  units = W.shape[1]
  a_out = np.zeros(units)
  print(a_out)

  for i in range(units):
    print(f"\n현재 유닛은 {i+1}번째")
    z = np.dot(a_in, W[i]) + b[i] # 0번째 행 = 0번째 input이 모든 유닛으로 가는 weight
    # z = np.dot(a_in, W[:,i]) + b[i]
    A = g(z)
    print(A)
    a_out[i] = A

  return a_out

dense0(a_in, W, b, g=sigmoid)

[0. 0. 0.]

현재 유닛은 1번째
[1.00000226]

현재 유닛은 2번째
[1.]

현재 유닛은 3번째
[1.]


<ipython-input-22-d48f8b48e384>:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  a_out[i] = A


array([1.00000226, 1.        , 1.        ])

### 🔍 왜 [1] vs [1.00000226]이 되었는가?

*   잘못된 방향의 곱셈
*   다만 파이썬의 브로드캐스팅 연산이 적용되어 numpy는 어떻게든 연산을 시도함.

--->  문제가 없는 것으로 착각할 수 있지만 약간의 수치 오류 혹은 비정상 결과를 야기할 수 있음.

